> # Joko Slamet
> 18050974030 | Ujian Tengah Semester Data Mining | S1 Pendidikan Teknologi Informasi

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB,BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import warnings
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv('../resource/xAPI-Edu-Data.csv')
data

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,F,Jordan,Jordan,MiddleSchool,G-08,A,Chemistry,S,Father,5,4,5,8,No,Bad,Above-7,L
476,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,F,Father,50,77,14,28,No,Bad,Under-7,M
477,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,S,Father,55,74,25,29,No,Bad,Under-7,M
478,F,Jordan,Jordan,MiddleSchool,G-08,A,History,F,Father,30,17,14,57,No,Bad,Above-7,L


In [3]:

def cleanstage(x):
    if x=='lowerlevel' or x=='L':
        return 0
    elif x=="MiddleSchool"or x=='M':
        return 1
    else:
        return 2
data.StageID=data.StageID.apply(lambda y:cleanstage(y))
data['gender']=data.gender.apply(lambda x: 0 if x=='M' else 1)
data.ParentschoolSatisfaction=data.ParentschoolSatisfaction.apply(lambda x: 1 if x=='Good' else 0)
data.StudentAbsenceDays=data.StudentAbsenceDays.apply(lambda x: 0 if x=="Under-7" else 1)
data.Semester=data.Semester.apply(lambda x: 0 if x=='F' else 1)# 0=F - 1=S
data.GradeID=data.GradeID.apply(lambda x: int(x[2:]))
data.Relation=data.Relation.apply(lambda x: 0 if x=='Father' else 1)
data.ParentAnsweringSurvey=data.ParentAnsweringSurvey.apply(lambda x: 1 if x=='Yes' else 0)
data.Class=data.Class.apply(lambda x:cleanstage(x))
data.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,0,KW,KuwaIT,0,4,A,IT,0,0,15,16,2,20,1,1,0,1
1,0,KW,KuwaIT,0,4,A,IT,0,0,20,20,3,25,1,1,0,1
2,0,KW,KuwaIT,0,4,A,IT,0,0,10,7,0,30,0,0,1,0
3,0,KW,KuwaIT,0,4,A,IT,0,0,30,25,5,35,0,0,1,0
4,0,KW,KuwaIT,0,4,A,IT,0,0,40,50,12,50,0,0,1,1


In [4]:
x=data.loc[:,['gender', 'PlaceofBirth', 'StageID', 'GradeID', 'SectionID', 'Topic', 'Semester', 'Relation', 'raisedhands','VisITedResources','AnnouncementsView','Discussion', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'StudentAbsenceDays']].values
y=data.loc[:,'Class'].values

In [5]:
x=pd.DataFrame(x)
y=pd.DataFrame(y)

In [6]:
#Label encoding of features
x=x.apply(LabelEncoder().fit_transform)
y=y.apply(LabelEncoder().fit_transform)

In [7]:
print(x)

     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14
0     0   4   0   1   0   7   0   0  15  16   2  18   1   1   0
1     0   4   0   1   0   7   0   0  20  20   3  23   1   1   0
2     0   4   0   1   0   7   0   0  10   7   0  28   0   0   1
3     0   4   0   1   0   7   0   0  29  24   5  33   0   0   1
4     0   4   0   1   0   7   0   0  35  43  12  46   0   0   1
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
475   1   3   1   5   0   2   1   0   5   4   5   7   0   0   1
476   1   3   1   5   0   5   0   0  40  66  14  26   0   0   0
477   1   3   1   5   0   5   1   0  44  63  25  27   0   0   0
478   1   3   1   5   0   6   0   0  29  17  14  50   0   0   1
479   1   3   1   5   0   6   1   0  32  14  23  55   0   0   1

[480 rows x 15 columns]


In [8]:
#Scaling
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [9]:
#Creating a test and training dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state = 0)

# DECISION TREE

In [10]:
warnings.filterwarnings('ignore')
skf = StratifiedKFold(n_splits = 3, shuffle=True, random_state=0)
dt = DecisionTreeClassifier()
params_dt = {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'max_depth': np.arange(0,15)}
dt = GridSearchCV(dt, param_grid=params_dt, scoring='accuracy', cv=skf)
dt = dt.fit(X,y)
print(dt.best_params_, dt.best_score_)


{'criterion': 'gini', 'max_depth': 10, 'splitter': 'random'} 0.7416666666666667


In [11]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print("Algoritma Decision Tree")
print( '\nClassification report\n' )
print(classification_report(y_test, y_pred))
print( '\nConfusion Matrix\n' )
print(confusion_matrix(y_test, y_pred),'\n')
print('Accuracy :','{:.2f}'.format(accuracy_score(y_test, y_pred)))
print('F-Score :','{:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
print('AUC:','{:.2f}'.format(metrics.auc(fpr, tpr)))

Algoritma Decision Tree

Classification report

              precision    recall  f1-score   support

           0       0.82      0.82      0.82        34
           1       0.72      0.72      0.72        71
           2       0.64      0.64      0.64        39

    accuracy                           0.72       144
   macro avg       0.73      0.73      0.73       144
weighted avg       0.72      0.72      0.72       144


Confusion Matrix

[[28  6  0]
 [ 6 51 14]
 [ 0 14 25]] 

Accuracy : 0.72
F-Score : 0.72
AUC: 0.55


# KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(2, 61), 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}
knn = GridSearchCV(knn, param_grid = params_knn, scoring='accuracy', cv = skf)
knn.fit(X,y)
print(knn.best_params_, knn.best_score_)

{'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'distance'} 0.7270833333333334


In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=43, weights='distance')
# Train Decision Tree Classifer
knn = knn.fit(X_train,y_train)
#Predict the response for test dataset
y_pred= knn.predict(X_test)
print("Algoritma K-Nearest Neighbor (KNN)")
print( '\nClassification report\n' )
print(classification_report(y_test, y_pred))
print( '\nConfusion Matrix\n' )
print(confusion_matrix(y_test, y_pred),'\n')
print('Accuracy :','{:.2f}'.format(accuracy_score(y_test, y_pred)))
print('F-Score :','{:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
print('AUC:','{:.2f}'.format(metrics.auc(fpr, tpr)))

Algoritma K-Nearest Neighbor (KNN)

Classification report

              precision    recall  f1-score   support

           0       0.77      0.97      0.86        34
           1       0.78      0.73      0.75        71
           2       0.74      0.64      0.68        39

    accuracy                           0.76       144
   macro avg       0.76      0.78      0.77       144
weighted avg       0.76      0.76      0.76       144


Confusion Matrix

[[33  1  0]
 [10 52  9]
 [ 0 14 25]] 

Accuracy : 0.76
F-Score : 0.76
AUC: 0.54


# SVM

In [14]:
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
svc = SVC()
params_svc = {'C': np.logspace(start = -35, stop=100, base=1.05), 'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['auto', 'scale']}
svc = GridSearchCV(svc, param_grid=params_svc, scoring='accuracy', cv=3)
svc.fit(X,y)
print(svc.best_params_, svc.best_score_)

{'C': 0.6078307435713777, 'gamma': 'auto', 'kernel': 'sigmoid'} 0.7104166666666667


In [15]:
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
c_SVC = np.logspace(start = 0, stop = 10, num = 100, base = 2 , dtype = 'float64')
param_grid_S = {'C': c_SVC}
clf_SVC = SVC(C=100.0, kernel='linear', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
          probability=False, tol=0.001, cache_size=200, class_weight=None, 
          verbose=0, max_iter=-1, decision_function_shape="ovr", random_state = 0)
svc=clf_SVC.fit(X_train,y_train)
y_pred= svc.predict(X_test)
print("Algoritma Support Vector Machine (SVM)")
print( '\nClassification report\n' )
print(classification_report(y_test, y_pred))
print( '\nConfusion Matrix\n' )
print(confusion_matrix(y_test, y_pred),'\n')
print('Accuracy :','{:.2f}'.format(accuracy_score(y_test, y_pred)))
print('F-Score :','{:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
print('AUC:','{:.2f}'.format(metrics.auc(fpr, tpr)))

Algoritma Support Vector Machine (SVM)

Classification report

              precision    recall  f1-score   support

           0       0.79      0.88      0.83        34
           1       0.76      0.72      0.74        71
           2       0.69      0.69      0.69        39

    accuracy                           0.75       144
   macro avg       0.75      0.76      0.75       144
weighted avg       0.75      0.75      0.75       144


Confusion Matrix

[[30  4  0]
 [ 8 51 12]
 [ 0 12 27]] 

Accuracy : 0.75
F-Score : 0.75
AUC: 0.53
